In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm

# Scenario: Creating a Set of Machine Learning Friendly Features from EHR Data to Predict Diabetes Onset

First we will load in the necessary data files

In [2]:
def load_data_for_file(filename):
    print(f"Loading data for {filename}")
    df = pd.concat([ # use pd.concat to append/concatenate the data for all states together into a single frame
        pd.read_parquet(f"https://dicbworkshops.s3.amazonaws.com/{output_dir}/parquet/{filename}") # use read_csv to load the data from each output directory
        for output_dir in tqdm(['output_hi', 'output_ma', 'output_tx', 'output_wa']) # loop over each output directory
    ])
    return df

In [3]:
# load in the conditions
conditions = load_data_for_file('conditions.parquet')
observations = load_data_for_file('observations.parquet')
medications = load_data_for_file('medications.parquet')
procedures = load_data_for_file('procedures.parquet')

Loading data for conditions.parquet


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.82s/it]


Loading data for observations.parquet


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:37<00:00,  9.44s/it]


Loading data for medications.parquet


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.69s/it]


Loading data for procedures.parquet


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:12<00:00,  3.07s/it]


## Filtering Out Patients with Diagnoses of Type-2 Diabetes
For this exercise, we are interested in filtering out all patients with a diagnosis of Type-2 diabetes \
We can filter these out based on the SNOMED code `44054006`

In [4]:
type2_patients = conditions.query('CODE == 44054006').sort_values(by=['PATIENT', 'START']).drop_duplicates(subset=['PATIENT', 'START'], keep='first')

In [5]:
# now we create a dictionary/lookup table to map each patient's ID to the date of their earliest Type 2 diagnosis
patient_diagnosis_dates = {
    row['PATIENT']: row['START']
    for _, row in type2_patients.iterrows()
}

In [6]:
conditions

,START,STOP,PATIENT,ENCOUNTER,SYSTEM,CODE,DESCRIPTION
0,2001-07-18,None,8f8229e6-00be-a033-bb16-42781f9d208a,249a2734-060e-3b6e-de43-c5c2c97e888b,SNOMED-CT,473461003,Educated to high school level (finding)
1,2001-07-18,None,8f8229e6-00be-a033-bb16-42781f9d208a,249a2734-060e-3b6e-de43-c5c2c97e888b,SNOMED-CT,160903007,Full-time employment (finding)
2,2011-07-25,None,8f8229e6-00be-a033-bb16-42781f9d208a,7570e7f5-b7a8-3848-0208-07e278d3754e,SNOMED-CT,161744009,Past pregnancy history of miscarriage (situation)
3,2014-08-13,2015-07-22,8f8229e6-00be-a033-bb16-42781f9d208a,366aac19-54b2-b4af-2b03-8d35c03ae2ba,SNOMED-CT,73595000,Stress (finding)
4,2014-12-03,2015-07-15,8f8229e6-00be-a033-bb16-42781f9d208a,a047e3c3-9e86-dd54-f026-e28929aa1e6b,SNOMED-CT,72892002,Normal pregnancy (finding)
...,...,...,...,...,...,...,...
47332,2024-12-31,2025-01-14,e4f7d897-4345-a297-0ae3-ff6fbe5a71aa,f9215af9-3736-0599-7ceb-4485b8af96e7,SNOMED-CT,80583007,Severe anxiety (panic) (finding)
47333,2025-01-07,2025-01-21,e4f7d897-4345-a297-0ae3-ff6fbe5a71aa,c9444afc-a873-a18c-1e49-eb3a92c4dc01,SNOMED-CT,314529007,Medication review due (situation)
47334,2025-01-14,2025-01-28,e4f7d897-4345-a297-0ae3-ff6fbe5a71aa,a68b03e2-9449-782b-47d4-da72c2cfecda,SNOMED-CT,423315002,Limited social contact (finding)
47335,2025-01-28,2025-02-04,e4f7d897-4345-a297-0ae3-ff6fbe5a71aa,8b0ae691-9e4c-b1ce-0cc2-da9f273db9f9,SNOMED-CT,314529007,Medication review due (situation)


In [7]:
# add simplified date columns to the observations, medications, and procedures 
observations_simplified = observations.assign(
    DATE_SIMPLE=lambda x: pd.to_datetime(x['DATE']).dt.date.astype('str')
)
medications_simplified = medications.assign(
    DATE_SIMPLE=lambda x: pd.to_datetime(x['START']).dt.date.astype('str')
)
procedures_simplified = procedures.assign(
    DATE_SIMPLE=lambda x: pd.to_datetime(x['START']).dt.date.astype('str')
)

## Filtering Out Post-diagnosis Conditions, Observations, Medications, and Procedures and Unifying Into a Shared Representation
First we need to filter out all EHR data from encounters that took place after the Type-2 diabetes diagnosis for the type 2 patients

In [12]:
def filter_data(df, patients, date_column='DATE_SIMPLE'):
    data_filtered = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        patient = row['PATIENT']
        date = row[date_column]
        if patient in patients and patients[patient] > date:
            data_filtered.append(row)
    return pd.DataFrame(data_filtered)

In [13]:
conditions_filtered = filter_data(conditions, patient_diagnosis_dates, 'START') 

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 182373/182373 [00:02<00:00, 76691.11it/s]


In [14]:
observations_subset = observations_simplified[observations_simplified['PATIENT'].isin(patient_diagnosis_dates)]
observations_filtered = filter_data(observations_subset, patient_diagnosis_dates)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 789669/789669 [00:10<00:00, 76111.16it/s]


In [15]:
medications_subset = medications_simplified[medications_simplified['PATIENT'].isin(patient_diagnosis_dates)]
medications_filtered = filter_data(medications_subset, patient_diagnosis_dates)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 55269/55269 [00:00<00:00, 72479.17it/s]


In [17]:
procedures_subset = procedures_simplified[procedures_simplified['PATIENT'].isin(patient_diagnosis_dates)]
procedures_filtered = filter_data(procedures_subset, patient_diagnosis_dates)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 108374/108374 [00:01<00:00, 75210.53it/s]


In [18]:
procedures_filtered

,START,STOP,PATIENT,ENCOUNTER,SYSTEM,CODE,DESCRIPTION,BASE_COST,REASONCODE,REASONDESCRIPTION,DATE_SIMPLE
39,2016-02-23T02:53:46Z,2016-02-23T03:46:34Z,e1418fe5-3ca2-2d35-9cd0-88ec20bce2d6,799c717a-401a-e1e7-2b4f-ab040641b2b6,SNOMED-CT,710824005,Assessment of health and social care needs (pr...,600.50,NaN,None,2016-02-23
40,2016-02-23T03:46:34Z,2016-02-23T04:02:52Z,e1418fe5-3ca2-2d35-9cd0-88ec20bce2d6,799c717a-401a-e1e7-2b4f-ab040641b2b6,SNOMED-CT,710841007,Assessment of anxiety (procedure),600.50,NaN,None,2016-02-23
41,2016-02-23T04:02:52Z,2016-02-23T04:28:16Z,e1418fe5-3ca2-2d35-9cd0-88ec20bce2d6,799c717a-401a-e1e7-2b4f-ab040641b2b6,SNOMED-CT,866148006,Screening for domestic abuse (procedure),600.50,NaN,None,2016-02-23
42,2016-02-23T04:28:16Z,2016-02-23T04:41:42Z,e1418fe5-3ca2-2d35-9cd0-88ec20bce2d6,799c717a-401a-e1e7-2b4f-ab040641b2b6,SNOMED-CT,171207006,Depression screening (procedure),600.50,NaN,None,2016-02-23
43,2016-02-23T04:41:42Z,2016-02-23T05:04:24Z,e1418fe5-3ca2-2d35-9cd0-88ec20bce2d6,799c717a-401a-e1e7-2b4f-ab040641b2b6,SNOMED-CT,171207006,Depression screening (procedure),600.50,NaN,None,2016-02-23
...,...,...,...,...,...,...,...,...,...,...,...
188353,2000-01-17T17:24:45Z,2000-01-17T18:12:42Z,66de5451-b446-367b-821d-265b4cfda04f,f653d6cc-e7d0-1170-f770-49aff5d33dbe,SNOMED-CT,710824005,Assessment of health and social care needs (pr...,463.95,NaN,None,2000-01-17
188354,2000-01-17T18:12:42Z,2000-01-17T18:34:51Z,66de5451-b446-367b-821d-265b4cfda04f,f653d6cc-e7d0-1170-f770-49aff5d33dbe,SNOMED-CT,710841007,Assessment of anxiety (procedure),463.95,NaN,None,2000-01-17
188355,2000-01-17T18:34:51Z,2000-01-17T19:08:18Z,66de5451-b446-367b-821d-265b4cfda04f,f653d6cc-e7d0-1170-f770-49aff5d33dbe,SNOMED-CT,866148006,Screening for domestic abuse (procedure),463.95,NaN,None,2000-01-17
188356,2000-01-17T19:08:18Z,2000-01-17T19:20:13Z,66de5451-b446-367b-821d-265b4cfda04f,f653d6cc-e7d0-1170-f770-49aff5d33dbe,SNOMED-CT,171207006,Depression screening (procedure),463.95,NaN,None,2000-01-17


Now we will label the EHR data for the type 2 and non-type 2 patients and unify everythin into a single event set

In [35]:
def get_unified_records(conditions, observations, medications, procedures):
    return pd.concat([
    conditions[['PATIENT', 'START', 'CODE', 'DESCRIPTION']].assign(
        EVENT_TYPE='CONDITION',
    ).rename(columns={'START': 'DATE'}),
    observations[['PATIENT', 'DATE_SIMPLE', 'CODE', 'DESCRIPTION']].assign(
        EVENT_TYPE='OBSERVATION',
    ).rename(columns={'DATE_SIMPLE': 'DATE'}),
    medications[['PATIENT', 'DATE_SIMPLE', 'CODE', 'DESCRIPTION']].assign(
        EVENT_TYPE='MEDICATION',
    ).rename(columns={'DATE_SIMPLE': 'DATE'}),
    procedures[['PATIENT', 'DATE_SIMPLE', 'CODE', 'DESCRIPTION']].assign(
        EVENT_TYPE='PROCEDURE',
    ).rename(columns={'DATE_SIMPLE': 'DATE'})
    ])

In [36]:
# starting with the Type 2 diabetes patients, we label and unify the data from medications, procedures, observations, and conditions into a single recordset
all_records_type2 = get_unified_records(conditions_filtered, observations_filtered, medications_filtered, procedures_filtered)

In [37]:
# now we will do the same for the non type2 patients
all_records_non_type2 = get_unified_records(
    conditions[~conditions['PATIENT'].isin(type2_patients['PATIENT'])],
    observations_simplified[~observations_simplified['PATIENT'].isin(type2_patients['PATIENT'])],
    medications_simplified[~medications_simplified['PATIENT'].isin(type2_patients['PATIENT'])],
    procedures_simplified[~procedures_simplified['PATIENT'].isin(type2_patients['PATIENT'])],
)

In [38]:
all_records_non_type2

,PATIENT,DATE,CODE,DESCRIPTION,EVENT_TYPE
0,8f8229e6-00be-a033-bb16-42781f9d208a,2001-07-18,473461003,Educated to high school level (finding),CONDITION
1,8f8229e6-00be-a033-bb16-42781f9d208a,2001-07-18,160903007,Full-time employment (finding),CONDITION
2,8f8229e6-00be-a033-bb16-42781f9d208a,2011-07-25,161744009,Past pregnancy history of miscarriage (situation),CONDITION
3,8f8229e6-00be-a033-bb16-42781f9d208a,2014-08-13,73595000,Stress (finding),CONDITION
4,8f8229e6-00be-a033-bb16-42781f9d208a,2014-12-03,72892002,Normal pregnancy (finding),CONDITION
...,...,...,...,...,...
212079,3df4eaa0-3234-0118-df9e-5cbe4659744e,2024-10-28,866148006,Screening for domestic abuse (procedure),PROCEDURE
212080,3df4eaa0-3234-0118-df9e-5cbe4659744e,2024-10-28,171207006,Depression screening (procedure),PROCEDURE
212081,3df4eaa0-3234-0118-df9e-5cbe4659744e,2024-10-28,171207006,Depression screening (procedure),PROCEDURE
212082,3df4eaa0-3234-0118-df9e-5cbe4659744e,2024-10-28,428211000124100,Assessment of substance use (procedure),PROCEDURE


## Now Like Good Machine Learning Practitioners, we will split our Type 2 and Non-Type 2 patients into Training and Hold-out sets


In [41]:
from sklearn.model_selection import train_test_split
type2_patients = all_records_type2['PATIENT'].unique()
non_type2_patients = all_records_non_type2['PATIENT'].unique()
labels = np.concatenate([np.ones(type2_patients.shape), np.zeros(non_type2_patients.shape)])
train_patients, test_patients, train_labels, test_labels = train_test_split(np.concatenate([type2_patients, non_type2_patients]), labels, test_size=0.2, stratify=labels)

In [44]:
# now split the records accordingly
all_records = pd.concat([all_records_type2, all_records_non_type2])
train_records = all_records[all_records['PATIENT'].isin(train_patients)]
test_records = all_records[all_records['PATIENT'].isin(test_patients)]

## Option 1: Bag of Labeled Clinical Encounters
The simplest feature representation we can create and test is a binary vector representation \
which encodes the occurence or lack-therof of different clinical encounters/events \
To construct this representation, we can use the scikit-learn package's `CountVectorizer` class

In [45]:
from sklearn.feature_extraction.text import CountVectorizer

In [79]:
vectorizer = CountVectorizer(
    binary=True,
    tokenizer=lambda x: x.split('|'),
    token_pattern=None,
    lowercase=False
)

The vectorizer expects a `'|'` pipe delimited string of coded encounters, \
so we will construct this representation now for our training patients

In [80]:
# condense the records into a pipe-delimited string of event tokens per patient, 
# where each token is of the form <EVENT_TYPE>::<CODE>
train_records_condensed = train_records.assign(
    EVENT_TOKEN=lambda x: x['EVENT_TYPE'] + '::' + x['CODE'].astype(str) + '|'
).groupby(['PATIENT'])['EVENT_TOKEN'].sum().reset_index()

In [81]:
train_data_final = train_records_condensed.assign(
    LABEL=lambda x: x['PATIENT'].isin(type2_patients).astype(int)
)

In [82]:
train_data_final

,PATIENT,EVENT_TOKEN,LABEL
0,0007008e-e6b0-8ad7-b5b8-c5d6c0c731f4,CONDITION::314529007|CONDITION::314529007|COND...,0
1,00127d79-8f35-5109-fe9b-961386b57f99,CONDITION::314529007|CONDITION::66383009|CONDI...,0
2,001d8f80-cfca-0bfe-8c98-3fcaaccd1655,CONDITION::224299000|CONDITION::446654005|COND...,0
3,0029cec6-9b3b-ef64-8361-3cef09ec8439,CONDITION::314529007|CONDITION::314529007|COND...,0
4,0033e953-ab8f-26c6-af87-39cdede3b388,CONDITION::160968000|CONDITION::224299000|COND...,0
...,...,...,...
3655,ffa278a3-bf3c-4437-fe60-a0d8d83979a7,CONDITION::160968000|CONDITION::473461003|COND...,0
3656,ffa95e2b-cbcd-96c9-a14f-2de40f6be332,CONDITION::224295006|CONDITION::714628002|COND...,0
3657,ffd9ad6d-3975-439a-d549-109f33e7a1f2,CONDITION::428251008|CONDITION::714628002|COND...,1
3658,fff1fcf6-208c-2cf7-b058-5fc63bbc22d1,CONDITION::224295006|CONDITION::422650009|COND...,1


## Now that we have constructed a Pipe Delimited Event Representation, We Can Vectorize

In [83]:
event_occurence_vectors = vectorizer.fit_transform(train_data_final['EVENT_TOKEN'])

In [84]:
event_occurence_vectors.shape

(3660, 1206)

## Now We Will Test the performance of this representation on the Training Set Using KFold Cross Validation

In [97]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix

In [98]:
kfold = StratifiedKFold(n_splits=5)
clf = RandomForestClassifier()

In [99]:
metrics = []
for i, (train_index, test_index) in tqdm(enumerate(kfold.split(event_occurence_vectors, train_data_final['LABEL'])), total=5):
    train_x, train_y = event_occurence_vectors[train_index], train_data_final['LABEL'].to_numpy()[train_index]
    test_x, test_y = event_occurence_vectors[test_index], train_data_final['LABEL'].to_numpy()[test_index]
    # fit the model on the training fold
    clf.fit(train_x, train_y)
    # evaluate the model on the validation fold
    preds = clf.predict(test_x)
    scores = clf.predict_proba(test_x)[:, 1]
    # get the AUROC
    fpr, tpr, _ = roc_curve(test_y, scores)
    auroc = auc(fpr, tpr)
    # get the confusion matrix
    cm = confusion_matrix(test_y, preds)
    # save the metrics
    metrics.append({
        'AUROC': auroc,
        'Precision': cm[1, 1] / cm[:, 1].sum(),
        'Recall': cm[1, 1] / cm[1].sum(),
        'Specificity': cm[0, 0] / cm[0].sum()
    })
pd.DataFrame(metrics)
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.60it/s]


,AUROC,Precision,Recall,Specificity
0,0.951837,1.0,0.772727,1.0
1,0.923096,1.0,0.711111,1.0
2,0.942245,1.0,0.777778,1.0
3,0.967217,1.0,0.777778,1.0
4,0.942585,1.0,0.688889,1.0


## The model appears to be performing well so far, but is that where the story ends, or does the plot thicken?
One thing that we can do is audit the way our current model is behaving, and what its predictions are based \
on, by looking at feature importance rankings. Here we inspect the top 50 important features

In [92]:
reverse_lookup = {
    value: key for key, value in vectorizer.vocabulary_.items()
}
top_50_indices = np.argsort(clf.feature_importances_)[-50:]
top50_features = [reverse_lookup[idx] for idx in top_50_indices]
top50_importances = clf.feature_importances_[top_50_indices]
top_features_df = pd.DataFrame({
    'FEATURE_NAME': top50_features,
    'FEATURE_IMPORTANCE': top50_importances
}).assign(
    CODE=lambda x: x['FEATURE_NAME'].str.split('::').apply(lambda pair: pair[1])
).merge(
    all_records[['CODE', 'DESCRIPTION']].drop_duplicates().astype({'CODE': str}),
    on='CODE',
).sort_values(by='FEATURE_IMPORTANCE', ascending=False)

In [93]:
top_features_df

,FEATURE_NAME,FEATURE_IMPORTANCE,CODE,DESCRIPTION
50,OBSERVATION::32623-1,0.082823,32623-1,Platelet mean volume [Entitic volume] in Blood...
49,OBSERVATION::785-6,0.052471,785-6,MCH [Entitic mass] by Automated count
48,OBSERVATION::8302-2,0.045711,8302-2,Body Height
47,OBSERVATION::32207-3,0.042240,32207-3,Platelet distribution width [Entitic volume] i...
46,OBSERVATION::718-7,0.038963,718-7,Hemoglobin
45,OBSERVATION::718-7,0.038963,718-7,Hemoglobin [Mass/volume] in Blood
44,OBSERVATION::72166-2,0.035195,72166-2,Tobacco smoking status
43,OBSERVATION::6690-2,0.032629,6690-2,Leukocytes [#/volume] in Blood by Automated count
42,OBSERVATION::4544-3,0.029721,4544-3,Hematocrit [Volume Fraction] of Blood by Autom...
41,OBSERVATION::786-4,0.027691,786-4,MCHC [Mass/volume] by Automated count
